## Variations on the Taxi-Grid Enviroment

### Motivation

Our project focuses on variations on the taxi gym environment (https://gym.openai.com/envs/Taxi-v3/), which is a basic "grid-world" enviroment used for research in artificial intelligence and reinforcement learning.

In [1]:
import gym

env = gym.make("Taxi-v3").env

env.reset ()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



This is original taxi grid enviroment, which is available as part of the OpenAI gym module. The enviroment models a taxi, which moves within the gridworld enviroment with the goal of picking up a passenger and bringing her to her destination.

Our modifications on the original enviroment were created with two main
goals: first, the original taxi grid enviroment is a single agent enviroment. We were interested in multi-agent reinforcement learning, so we created a variation on the enviroment with multiple taxis. Second, by adding a fuel constraint modification, we were able to test whether agents would be able consider the long term fuel constraint. Here, our goal was to make the enviroment more flexible and dynamic by adding a variation with fuel constraints, the ability to use custom grids, and the ability to have an arbitrary number of passengers.

### Installation

To work with our custom enviroments, the first thing that we want to do is to install them. As the enviroments are packaged together into a module called "multitaxienv" which is to be found on the github, it is easy to install all of the modules at once.

In [2]:
!pip install git+https://github.com/sarah-keren/MutliTaxiEnv.git --upgrade

  Cloning https://github.com/sarah-keren/MutliTaxiEnv.git to /tmp/pip-req-build-r3uofinr
  Running command git clone -q https://github.com/sarah-keren/MutliTaxiEnv.git /tmp/pip-req-build-r3uofinr
  Created wheel for MultiTaxiEnv: filename=MultiTaxiEnv-0.2-cp36-none-any.whl size=11042 sha256=4276509999855777277fc12b9947e5e0959f932039121f2ca297a1ffff03d204
  Stored in directory: /tmp/pip-ephem-wheel-cache-1u1ngwdm/wheels/a7/8b/8f/2c7828f791585f08f092f39fb14023ccf64d14a1a3d68143ef
Successfully built MultiTaxiEnv


Now, once we have installed the module, we are able to import the three enviroments, OneTaxiFuelEnv, which is the single taxi with fuel enviroment, MultiTaxiEnv, which is the multiple taxi without fuel enviroment, and MultiTaxiFuelEnv, which is the multiple taxi with fuel enviroment.

In [3]:
from multitaxienv.onetaxifuel_env import OneTaxiFuelEnv
from multitaxienv.multitaxi_env import MultiTaxiEnv
from multitaxienv.multitaxifuel_env import MultiTaxiFuelEnv

### Single Taxi With Fuel

First, we will demonstrate the most basic enviroment - one taxi with fuel.

In [4]:
env = OneTaxiFuelEnv()
env.reset()
env.s = 1022
env.render()

+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| : |X: |
+---------+
Fuel: 10



TaxiFuelEnv is an enviroment where there is one taxi. The taxi is represented by the yellow highlighted block. The objective of the taxi in this enviroment is to move the passenger from the blue X to the magenta X. The action space of the taxi is (0,1,2,3,4,5,6), where 0,1,2,3 are move south, north, east, west respectively, 4 is pickup passenger, 5 is dropoff passenger, and 6 is refuel.

In [5]:
state, reward, done, _ = env.step(6)
print("The next state is: " + str(state) + ", the reward for the last action is: " + str(reward) + ", and the episode is "+ {True: "", False: "not"}[done]  + " done.")

The next state is: 1022, the reward for the last action is: -10, and the episode is not done.


To preform an action, we use the step function. This returns a tuple which includes the next state, the reward, whetheer the episode has ended or not, and debugging info which we don't need to worry about for our purposes. In this environment, the episode ends when the passenger is dropped off successfully. The reward for each step is -1, unless there is an illegal pickup, dropoff, or refuel whch yields reward of -10, and a successful dropoff gives reward +20 (all these values are configurable in config.py). Taxis can only dropoff passengers at their final destination and nowhere else.

Now, we will navigate to the fuel station by moving west twice.

In [6]:
env.step(3)
env.render()
env.step(3)
env.render()

+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| : |X: |
+---------+
Fuel: 9
  (West)
+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| : |X: |
+---------+
Fuel: 8
  (West)


Here, note that since we have moved two steps, we have consumed two units of fuel. There is a large penalty for moving when the taxi does not have fuel. We will now refuel.

In [7]:
state, reward, done, _ = env.step(6)
env.render()

+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| : |X: |
+---------+
Fuel: 8
  (Refill)


Here, since we have refuelled, we are back at full. Another thing to note is that the state of the environment is encoded using a single number that represents the state. However, sometimes it may be useful to decode what that number actually means (for example, when using deep-Q learning). We can do that using the decode function.

In [8]:
x, y, pass_loc, pass_dest, fuel = list(env.decode(state))
print("The coordinates of the taxi are currently: " + "(" + str(x) + "," + str(y) +")")
print("The index of the passenger location is: " + str(pass_loc) + ", while the index of the passenger destination is: " + str(pass_dest))
print("Currently, the fuel level of the taxi is: " + str(fuel))

The coordinates of the taxi are currently: (0,2)
The index of the passenger location is: 3, while the index of the passenger destination is: 0
Currently, the fuel level of the taxi is: 8


You can also input your own map as a list of strings and specify the maximum fuel. The map must be formatted in the same way as above, with '+'at the corners and '-', '|' specifying the boundaries. You can have as many destination and fuel stations as you want!

In [9]:
custom_map = [
    '+---------------+',
    '| : :X| :F: : : |',
    '|X: : | : | :X| |',
    '| : : : : : : | |',
    '| :X:F| :X| : :X|',
    '+---------------+',
]

In [10]:
env = OneTaxiFuelEnv(max_fuel=6, map=custom_map)
env.reset()
env.render()

+---------------+
| : :X| :F: : : |
|X: : | : | :X| |
| : : : : : : | |
| :X:F| :X| : :X|
+---------------+
Fuel: 6



This concludes the demonstration for the single taxi with fuel environment.

### Multiple taxis without fuel

This environment supports having multiple taxis and multiple passengers which you can specify when initializing. The default initialization is two taxis and one passenger. The goal is to deliver all passengers to their destinations. Each taxi can only carry one passenger at a time, but can dropoff passengers anywhere.

In [11]:
env = MultiTaxiEnv(num_taxis=3, num_passengers=2)
env.reset()
env.state = [[[2, 2], [0, 1], [2, 1]], [[4, 0], [0, 0]], [[0, 0], [4, 3]], [0, 0]]
env.render()

+---------+
|X:_| : :X|
| : | : : |
| :_:_: : |
| | : | : |
|X| : |X: |
+---------+
Taxi1: Location: (2,2)
Taxi2: Location: (0,1)
Taxi3: Location: (2,1)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: (0, 0), Destination: (4, 3)


Each state is recorded as a list of lists:
1. a list of locations of each taxi, formatted [row, col]
2. a list of passenger starting locations as coordinates
3. a list of passenger destinations as coordinates
4. a list of current passenger locations as integers, 0 means not picked up, -1 means reached destination, and positive number specifies which taxi they are in

The i'th observation in each of the relevant passenger lists correspond to the same passenger.



In [12]:
print(env.state)

[[[2, 2], [0, 1], [2, 1]], [[4, 0], [0, 0]], [[0, 0], [4, 3]], [0, 0]]


Actions are inputted as list of actions corresponding to each taxi. Each taxi has the same actions as in the above environment with only one taxi except for refueling. Thus the action space of the taxi is (0,1,2,3,4,5,6), where 0,1,2,3 are move south, north, east, west respecitvely, 4 is pickup passenger, 5 is dropoff passenger, and 6 is standby.

Let's move taxis 1 and 3 south, and taxi 2 west.

In [13]:
state, reward, done, _ = env.step([0,3,0])
print("Now the reward is given as the reward for each individual taxi: " + str(reward))

env.render()

Now the reward is given as the reward for each individual taxi: [-1, -1, -1]
+---------+
|X: | : :X|
| : | : : |
| : : : : |
| |_:_| : |
|X| : |X: |
+---------+
  (South ,West ,South)
Taxi1: Location: (3,2)
Taxi2: Location: (0,0)
Taxi3: Location: (3,1)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: (0, 0), Destination: (4, 3)


To demonstrate the ability to dropoff passengers anywhere, we will pickup passenger 2 with taxi 2 while telling other taxis to dropoff. Then we will move taxi 2 south twice and dropoff.

In [14]:
actions = [[5,4,5],[5,0,5],[5,0,5],[5,5,5]]
for action in actions:
    env.step(action)
    env.render()

+---------+
|X: | : :X|
| : | : : |
| : : : : |
| |_:_| : |
|X| : |X: |
+---------+
  (Dropoff ,Pickup ,Dropoff)
Taxi1: Location: (3,2)
Taxi2: Location: (0,0)
Taxi3: Location: (3,1)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: Taxi2, Destination: (4, 3)
+---------+
|X: | : :X|
| : | : : |
| : : : : |
| |_:_| : |
|X| : |X: |
+---------+
  (Dropoff ,South ,Dropoff)
Taxi1: Location: (3,2)
Taxi2: Location: (1,0)
Taxi3: Location: (3,1)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: Taxi2, Destination: (4, 3)
+---------+
|X: | : :X|
| : | : : |
| : : : : |
| |_:_| : |
|X| : |X: |
+---------+
  (Dropoff ,South ,Dropoff)
Taxi1: Location: (3,2)
Taxi2: Location: (2,0)
Taxi3: Location: (3,1)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: Taxi2, Destination: (4, 3)
+---------+
|X: | : :X|
| : | : : |
| : : : : |
| |_:_| : |
|X| : |X: |
+---------+
  (Dropoff ,Dropoff ,Dropoff)
Taxi1: Location: (3,2)
Taxi2: Location: (2,0)

We can see that passenger 2's location was taxi 2, and at the end they are at the new location (2,0). Now let's  pick them up with taxi 3.

In [15]:
actions = [[5,5,1],[5,5,3],[5,5,4],[5,5,2]]
for action in actions:
    env.step(action)
    env.render()

+---------+
|X: | : :X|
| : | : : |
| :_: : : |
| | :_| : |
|X| : |X: |
+---------+
  (Dropoff ,Dropoff ,North)
Taxi1: Location: (3,2)
Taxi2: Location: (2,0)
Taxi3: Location: (2,1)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: (2, 0), Destination: (4, 3)
+---------+
|X: | : :X|
| : | : : |
| : : : : |
| | :_| : |
|X| : |X: |
+---------+
  (Dropoff ,Dropoff ,West)
Taxi1: Location: (3,2)
Taxi2: Location: (2,0)
Taxi3: Location: (2,0)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: (2, 0), Destination: (4, 3)
+---------+
|X: | : :X|
| : | : : |
| : : : : |
| | :_| : |
|X| : |X: |
+---------+
  (Dropoff ,Dropoff ,Pickup)
Taxi1: Location: (3,2)
Taxi2: Location: (2,0)
Taxi3: Location: (2,0)
Passenger1: Location: (4, 0), Destination: (0, 0)
Passenger2: Location: Taxi3, Destination: (4, 3)
+---------+
|X: | : :X|
| : | : : |
|_: : : : |
| | :_| : |
|X| : |X: |
+---------+
  (Dropoff ,Dropoff ,East)
Taxi1: Location: (3,2)
Taxi2: Location: (2,0)
T

Now taxi 3 has picked up passenger 2 and can transport them around. This simple example illustrates this new function that wasn't available in the single taxi environment where the taxi could only dropoff at the destination.

We can also  choose to input our own custom map as well.

In [16]:
custom_map = [
    '+---------------+',
    '| : :X| : : : : |',
    '|X: : | : | :X| |',
    '| : : : : : : | |',
    '| :X: | :X| : :X|',
    '+---------------+',
]

In [17]:
env = MultiTaxiEnv(map=custom_map)
env.reset()
env.render()

+---------------+
| : :X| : : : : |
|X: : | : | :X| |
| : :_: : : : | |
| :X: | :X| : :X|
+---------------+
Taxi1: Location: (2,2)
Taxi2: Location: (3,7)
Passenger1: Location: (3, 1), Destination: (3, 4)


Below we will explore the most interesting environment.

### Multiple taxi with fuel

The most complex environment is the multiple taxi with fuel environment. This environment supports an arbitrary number of taxis with fuel and an arbitrary number of passengers. By default, the environment is initialized with two taxis, one passenger, and the maximum (and starting) fuel of each taxi is 8. However, we are able to change all of those parameters like above. This is essentially just the MultiTaxi environment with fuel constraints, so we are still able to dropoff passengers anywhere.

In [18]:
env = MultiTaxiFuelEnv(num_taxis=2, num_passengers=2, max_fuel=8)
env.reset()
env.render()

+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | :_|
|X| :G|X: |
+---------+
Taxi1: Fuel: 8, Location: (0,0)
Taxi2: Fuel: 8, Location: (3,4)
Passenger1: Location: (4, 0), Destination: (4, 3)
Passenger2: Location: (0, 4), Destination: (4, 3)


Initializing the environment, we have the location and fuel values for each of the passneger, as well as the location and destination values for each passenger. In the multiple passenger enviroment, the episode does not end until each passenger is delivered to their destination.

Each state is recorded as a list of lists like the MultiTaxi environment:
1. a list of locations of each taxi, formatted [row, col]
2. a list of integers specifying fuel levels for each taxi
2. a list of passenger starting locations as coordinates
3. a list of passenger destinations as coordinates
4. a list of current passenger locations as integers, 0 means not picked up, -1 means reached destination, and positive number specifies which taxi they are in

In [19]:
print(env.state)

[[[0, 0], [3, 4]], [8, 8], [[4, 0], [0, 4]], [[4, 3], [4, 3]], [0, 0]]


Like the MultiTaxi environment, actions are now lists of individual actions,  and the actions for each individual taxi are the same as in the original one taxi fuel enviroment (8 poassible actions). For example, suppose that I wish for taxi 1 to go north (action 1) and for taxi 2 to go south (action 0).

In [20]:
state, reward, done, _ = env.step([1,0])
print("Now the reward is given as the reward for each individual taxi: " + str(reward))

env.render()

Now the reward is given as the reward for each individual taxi: [-20, -1]
+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| :G|X:_|
+---------+
  (North ,South)
Taxi1: Fuel: 8, Location: (0,0)
Taxi2: Fuel: 7, Location: (4,4)
Passenger1: Location: (4, 0), Destination: (4, 3)
Passenger2: Location: (0, 4), Destination: (4, 3)


Now, we will try a random solution, and see how long it takes to finish an episode. In particular, at every time step, each of the taxis will choose a random action. We can easily do this using the env.action_space.sample() function, which returns a random sample of the action space corresponding to a random action by each taxi.

In [21]:
env.reset()
epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward[0] == -10 or reward[1] == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 16855
Penalties incurred: 15645


Here, as we have seen, the episode takes a very long time to finish, and the taxis incur many penalties, meaning that they have tried many invalid moves. We can see this in action by replaying the frames of that episode.

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|X: |F: :X|
| : | : :_|
| : : : : |
| | : | : |
|X| :G|X: |
+---------+
  (East ,North)
Taxi1: Fuel: 0, Location: (0,4)
Taxi2: Fuel: 0, Location: (1,4)
Passenger1: Location: (0, 2), Destination: (4, 0)
Passenger2: Location: (2, 1), Destination: (4, 3)

Timestep: 3885
State: [[[3, 3], [4, 3]], [0, 0], [[2, 1], [3, 3]], [[4, 0], [4, 3]], [-1, -1]]
Action: [2 1]
Reward: [-20, -10]
